In [ ]:
import numpy as np
import pandas as pd
import pickle
import torch.nn as nn
from torch.utils.data import DataLoader
import json, ast, sys, csv, random
import plotly.express as px
import math
import datetime
import sys
import csv
import numpy as np
import matplotlib.pyplot as plt


#Implement training process
from model_trees_algebra import NeoRegression

from sklearn.utils import shuffle
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

from functions.tree_format import IterateBuildTree, InnerJoinsIntraBGPS, \
                                IterateBuildTreeBetweenBGPS, TreeFormat
from functions.RL_functions import GetTriplesSubtree, \
                                        GetTreeSize, \
                                        GetAllJoins, \
                                        GetIter, \
                                        GetTotalBgp, \
                                        GetDataframe

from functions.aux import MetricTotalAccuraccy


class BaoTrainingException(Exception):
    pass
csv.field_size_limit(sys.maxsize)
URL = "/media/data/ccarmona/memoria/dataset/rl_csvs/"

In [ ]:
1+1

In [ ]:
#ds_rl = pd.read_csv(URL + "ds_rl.csv", engine='python', encoding='utf-8')
n = 4
#neo_df = pd.read_csv(URL + "neo_df.csv", engine='python', encoding='utf-8')
neo_df_no_history = pd.read_csv(URL + "neo_df_no_history" + str(n) +".csv", engine='python', encoding='utf-8')
neo_df_final = pd.read_csv(URL + "neo_df_final"+ str(n) +".csv", engine='python', encoding='utf-8')

#ql_df = pd.read_csv(URL + "ql_df.csv", engine='python', encoding='utf-8')
ql_df_no_history = pd.read_csv(URL + "ql_df_no_history"+ str(n) +".csv", engine='python', encoding='utf-8')
ql_df_final = pd.read_csv(URL + "ql_df_final"+ str(n) +".csv", engine='python', encoding='utf-8')

#sarsa_df = pd.read_csv(URL + "sarsa_df.csv", engine='python', encoding='utf-8')
sarsa_df_no_history = pd.read_csv(URL + "sarsa_df_no_history"+ str(n) +".csv", engine='python', encoding='utf-8')
sarsa_df_final = pd.read_csv(URL + "sarsa_df_final"+ str(n) +".csv", engine='python', encoding='utf-8')

#exsarsa_df = pd.read_csv(URL + "exsarsa_df.csv", engine='python', encoding='utf-8')
exsarsa_df_no_history = pd.read_csv(URL + "exsarsa_df_no_history"+ str(n) +".csv", engine='python', encoding='utf-8')
exsarsa_df_final = pd.read_csv(URL + "exsarsa_df_final"+ str(n) +".csv", engine='python', encoding='utf-8')

In [ ]:
neo_df_final

In [ ]:
ql_df_final

In [ ]:
sarsa_df_final

In [ ]:
exsarsa_df_final

### Latencia promedio de cada algoritmo + promedio real

In [ ]:
"""
========
Barchart
========

A bar plot with errorbars and height labels on individual bars
"""
import numpy as np
import matplotlib.pyplot as plt
plt.rcParams.update({'font.size': 13})
ind = np.arange(4)*0.4
real_value = [np.mean(ql_df_final['real_new']) for i in range(4)]
old_pred = [np.mean(ql_df_final['pred_old']) for i in range(4)]
h = [
    np.mean(neo_df_final['pred_new']),
    np.mean(ql_df_final['pred_new']),
    np.mean(sarsa_df_final['pred_new']),
    np.mean(exsarsa_df_final['pred_new']),
]
c = ['lightgreen','lightblue','yellow','orange']
names = ('Neo','Q-Learning','Sarsa','Expected Sarsa')

width = 0.2      # the width of the bars

fig, ax = plt.subplots(figsize=(15,15))
rects = ax.bar(ind, h, width, color=c, edgecolor = 'black')
line1 = ax.plot(ind, real_value);
line2 = ax.plot(ind, old_pred);

# add some text for labels, title and axes ticks
ax.set_ylabel('Latencia')
ax.set_title('Promedio de Latencia por Algoritmo de RL')
ax.set_xticks(ind)
ax.set_xticklabels(names)


def autolabel(rects):
    """
    Attach a text label above each bar displaying its height
    """
    for rect in rects:
        height = rect.get_height()
        ax.text(rect.get_x() + rect.get_width()/2., 0.75*height,
                str(round(float(height),3)),
                ha='center', va='bottom')

ax.text(ind[1]+0.05,real_value[0] + 0.55,'Promedio de latencia real')
ax.text(ind[1]+0.05,old_pred[0] + 0.55,'Promedio de latencia red neuronal')
ax.text(ind[3]+0.01,real_value[0] - 0.1,str(round(real_value[0],3)))
ax.text(ind[3]+0.01,old_pred[0] - 0.1,str(round(old_pred[0],2)))
autolabel(rects)


plt.show()

### Latencia promedio por iteración de  cada consulta para cada algoritmo + promedio real

In [ ]:
def PromedioPorIteracionFormato(df, num_iter):
    aux_dict = {}
    values_pred = []
    for i in range(num_iter):
        aux_dict[str(i)] = {"index" : 0, "preds" : [], "mean_preds" : []}
    for i in range(len(df)):
        a = [df['index'][i],df['iteration'][i],df['pred_new'][i]]
        aux_dict[str(a[1])]["index"] = a[0]
        aux_dict[str(a[1])]["preds"].append(float(a[2]))
    for k,v in aux_dict.items():
        aux_dict[k]["mean_preds"] = np.mean(aux_dict[k]["preds"])
        values_pred.append(np.mean(aux_dict[k]["preds"]))
    index_list = [int(x) for x in list(aux_dict.keys())]
    return index_list, values_pred
def PromedioPorIteracion(df):
    index_list, values_pred = PromedioPorIteracionFormato(df)
    fig = plt.figure()
    ax = plt.axes()
    ax.plot(index_list, values_pred);

In [ ]:
index_list_ql, values_pred_ql = PromedioPorIteracionFormato(ql_df_no_history,200)
index_list_neo, values_pred_neo = PromedioPorIteracionFormato(neo_df_no_history,200)
index_list_sarsa, values_pred_sarsa = PromedioPorIteracionFormato(sarsa_df_no_history,200)
index_list_exsarsa, values_pred_exsarsa = PromedioPorIteracionFormato(exsarsa_df_no_history,200)

In [ ]:
real_value = np.ones(len(index_list_ql))*np.mean(ql_df_final['real_new'])
old_predict = np.ones(len(index_list_ql))*np.mean(ql_df_final['pred_old'])

In [ ]:
np.mean(ql_df_final['pred_old'])

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Some example data to display

fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(15,15))
fig.suptitle('Promedio de latencia por iteración para cada algoritmo')
c = ['green','blue','yellow','orange','grey','black']
ax1.set_title('Neo')
ax1.plot(index_list_neo, values_pred_neo, color=c[0])
#ax1.plot(index_list_neo, real_value, color=c[5], linestyle='dashed')
ax1.plot(index_list_neo, old_predict, color=c[4], linestyle='dashed')

ax2.set_title('Q-Learning')
ax2.plot(index_list_ql, values_pred_ql, color=c[1])
#ax2.plot(index_list_ql, real_value, color=c[5], linestyle='dashed')
ax2.plot(index_list_ql, old_predict, color=c[4], linestyle='dashed')

ax3.set_title('Sarsa')
ax3.plot(index_list_sarsa, values_pred_sarsa, color=c[2])
#ax3.plot(index_list_sarsa, real_value, color=c[5], linestyle='dashed')
ax3.plot(index_list_sarsa, old_predict, color=c[4], linestyle='dashed')

ax4.set_title('Expected Sarsa')
ax4.plot(index_list_exsarsa, values_pred_exsarsa, color=c[3])
#ax4.plot(index_list_exsarsa, real_value, color=c[5], linestyle='dashed')
ax4.plot(index_list_exsarsa, old_predict, color=c[4], linestyle='dashed')
#for ax in fig.get_axes():
#ax.label_outer()

# set labels
plt.setp(ax1, xlabel='Iteraciones', ylabel='Latencia promedio')
plt.setp(ax2, xlabel='Iteraciones', ylabel='Latencia promedio')
plt.setp(ax3, xlabel='Iteraciones', ylabel='Latencia promedio')
plt.setp(ax4, xlabel='Iteraciones', ylabel='Latencia promedio')


### Consultas mejoradas, consultas iguales, consultas empeoradas

In [ ]:
def tag_points(x):
    """Add quality tags of predictions. Used to plot with plotly"""
    difference = x['time'] - x['y_pred'][0]
    abs_diff = np.abs(difference)
    x['y_pred'] = x['y_pred'][0]
    x['query2'] = x['query'].replace(" . ", ' . <br>').replace(" FILTER", '<br> FILTER').replace(" { ", ' { <br>').replace(" } ", ' <br> }').replace(" ; ", ' ; <br>') 
    p20 = x['time'] * 0.2
    p40 = x['time'] * 0.4
    if abs_diff < p20:
        x['color'] = "good prediction"
    elif abs_diff < p40:
        x['color'] = "aceptable prediction"
    else:
        x['color'] = "bad prediction"
    return x

In [ ]:
def tag_best_worst_equal(x, tol):
    difference = x['pred_old'] - x['pred_new']
    if difference < -tol:
        x['tag'] = "peor"
    elif difference >= -tol and difference <= tol:
        x['tag'] = "igual"
    else:
        x['tag'] = "mejor"
    
    return x

In [ ]:
neo_df_final = neo_df_final.apply(lambda x: tag_best_worst_equal(x,0), axis=1)
ql_df_final = ql_df_final.apply(lambda x: tag_best_worst_equal(x,0), axis=1)
sarsa_df_final = sarsa_df_final.apply(lambda x: tag_best_worst_equal(x,0), axis=1)
exsarsa_df_final = exsarsa_df_final.apply(lambda x: tag_best_worst_equal(x,0), axis=1)

In [ ]:
#for i in range(len(ql_df_final)):
#    print(neo_df_final['pred_old'][i],neo_df_final['pred_new'][i],neo_df_final['tag'][i])

In [ ]:
neo_igual = int(neo_df_final.groupby('tag').count()['index']['igual'])
neo_mejor = int(neo_df_final.groupby('tag').count()['index']['mejor'])
neo_peor = int(neo_df_final.groupby('tag').count()['index']['peor'])

ql_igual = int(ql_df_final.groupby('tag').count()['index']['igual'])
ql_mejor = int(ql_df_final.groupby('tag').count()['index']['mejor'])
ql_peor = int(ql_df_final.groupby('tag').count()['index']['peor'])

sarsa_igual = int(sarsa_df_final.groupby('tag').count()['index']['igual'])
sarsa_mejor = int(sarsa_df_final.groupby('tag').count()['index']['mejor'])
sarsa_peor = int(sarsa_df_final.groupby('tag').count()['index']['peor'])

exsarsa_igual = int(exsarsa_df_final.groupby('tag').count()['index']['igual'])
exsarsa_mejor = int(exsarsa_df_final.groupby('tag').count()['index']['mejor'])
exsarsa_peor = int(exsarsa_df_final.groupby('tag').count()['index']['peor'])

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
plt.rcParams.update({'font.size': 13})
# data to plot
n_groups = 4
igual = (neo_igual, ql_igual, sarsa_igual, exsarsa_igual)
mejor = (neo_mejor, ql_mejor, sarsa_mejor, exsarsa_mejor)
peor = (neo_peor, ql_peor, sarsa_peor, exsarsa_peor)

# create plot
fig, ax = plt.subplots(figsize=(15,10))
index = np.arange(n_groups)
bar_width = 0.25
opacity = 0.8

rects1 = plt.bar(index, mejor, bar_width,
alpha=opacity,
color='lightgreen',
label='Mejor',
edgecolor = 'black')

rects2 = plt.bar(index + bar_width, igual, bar_width,
alpha=opacity,
color='khaki',
label='Igual',
edgecolor = 'black')

rects3 = plt.bar(index + 2*bar_width, peor, bar_width,
alpha=opacity,
color='darksalmon',
label='Peor',
edgecolor = 'black')

plt.xlabel('Algorítmo')
plt.ylabel('Número de Errores')
plt.title('Número de errores por Algoritmo')
plt.xticks(index + bar_width, ('Neo', 'Q-Learning', 'Sarsa', 'Expected Sarsa'))
plt.legend()
plt.tight_layout()


for i in index:
    ax.text(index[i], 
             mejor[i]*0.75,
             str(mejor[i]),
             ha='center', va='bottom',
            )
    ax.text(index[i] + bar_width, 
             igual[i]*0.75,
             str(igual[i]),
             ha='center', va='bottom',
            )
    ax.text(index[i] + 2*bar_width, 
             peor[i]*0.75,
             str(peor[i]),
             ha='center', va='bottom',
            )


plt.show()

##### Por tipo rango de latencia

In [ ]:
def split_by_latency(df_final):
    df = df_final.copy()
    #df_0_10 = df[df['pred_new'] < 10]
    #df_10_20 = df[(df['pred_new'] >= 10) & (df['pred_new'] < 20)]
    #df_20_30 = df[(df['pred_new'] >= 20) & (df['pred_new'] < 30)]
    #df_30_40 = df[(df['pred_new'] >= 30) & (df['pred_new'] < 40)]
    #df_40_50 = df[(df['pred_new'] >= 40) & (df['pred_new'] < 50)]
    #df_50_60 = df[(df['pred_new'] >= 50) & (df['pred_new'] < 60)]
    #df_60 = df[df['pred_new'] >= 60]
    df_0_20 = df[df['real_old'] < 20]
    df_20_30 = df[(df['real_old'] >= 20) & (df['real_old'] < 30)]
    df_30_40 = df[(df['real_old'] >= 30) & (df['real_old'] < 40)]
    df_40_50 = df[(df['real_old'] >= 40) & (df['real_old'] < 50)]
    df_50_60 = df[(df['real_old'] >= 50) & (df['real_old'] < 60)]
    df_60 = df[df['real_old'] >= 60]
    #return df_0_10, df_10_20 ,df_20_30, df_30_40, df_40_50, df_50_60, df_60
    return df_0_20,df_20_30,df_30_40,df_40_50,df_50_60,df_60

def count_igual_mejor_peor(df):
    try:
        igual = int(df.groupby('tag').count()['index']['igual'])
    except:
        igual = 0
    try:
        mejor = int(df.groupby('tag').count()['index']['mejor'])
    except:
        mejor = 0
    try:
        peor = int(df.groupby('tag').count()['index']['peor'])
    except:
        peor = 0
    return igual,mejor,peor

In [ ]:
def plot_errores_rango(df_final,nombre_algoritmo):
    igual = []
    mejor = []
    peor = []
    for df in split_by_latency(df_final):
        i, m, p = count_igual_mejor_peor(df)
        igual.append(i)
        mejor.append(m)
        peor.append(p)
    plt.rcParams.update({'font.size': 13})
    # data to plot
    n_groups = len(igual)
    #igual = (neo_igual, ql_igual, sarsa_igual, exsarsa_igual)
    #mejor = (neo_mejor, ql_mejor, sarsa_mejor, exsarsa_mejor)
    #peor = (neo_peor, ql_peor, sarsa_peor, exsarsa_peor)

    # create plot
    fig, ax = plt.subplots(figsize=(15,10))
    index = np.arange(n_groups)
    bar_width = 0.15
    opacity = 0.8

    rects1 = plt.bar(index, mejor, bar_width,
    alpha=opacity,
    color='lightgreen',
    label='Mejor',
    edgecolor = 'black')

    rects2 = plt.bar(index + bar_width, igual, bar_width,
    alpha=opacity,
    color='khaki',
    label='Igual',
    edgecolor = 'black')

    rects3 = plt.bar(index + 2*bar_width, peor, bar_width,
    alpha=opacity,
    color='darksalmon',
    label='Peor',
    edgecolor = 'black')

    plt.xlabel('Rango de latencia')
    plt.ylabel('Número de Errores')
    plt.title(f'Número de errores {nombre_algoritmo} por rangos de latencia')
    plt.xticks(index + bar_width, ('[0-20)','[20-30)','[30-40)','[40-50)','[50-60)','60+'))
    plt.legend()
    plt.tight_layout()


    for i in index:
        ax.text(index[i], 
                 mejor[i]*0.75,
                 str(mejor[i]),
                 ha='center', va='bottom',
                )
        ax.text(index[i] + bar_width, 
                 igual[i]*0.75,
                 str(igual[i]),
                 ha='center', va='bottom',
                )
        ax.text(index[i] + 2*bar_width, 
                 peor[i]*0.75,
                 str(peor[i]),
                 ha='center', va='bottom',
                )
    plt.show()

In [ ]:
plot_errores_rango(neo_df_final,'Neo')
plot_errores_rango(ql_df_final,'Q-Learning')
plot_errores_rango(sarsa_df_final,'Sarsa')
plot_errores_rango(exsarsa_df_final,'Expected Sarsa')

### Latencia promedio de cada consulta diferenciado por numero de JOINs para cada algoritmo

In [ ]:
def split_by_joins(df, val):
    joins_1_p, joins_1_e =[],[]
    joins_2_p, joins_2_e =[],[]
    joins_3_p, joins_3_e =[],[]
    joins_4_p, joins_4_e =[],[]
    joins_5_p, joins_5_e =[],[]
    joins_6_p, joins_6_e =[],[]
    #<
    #>
    columns = 'tag'
    for i in range(len(df)):
        count_joins = df['new_tree'][i].count('JOIN') 
        if count_joins == 1:
            joins_1_p.append(df[val][i])
            joins_1_e.append(df['tag'][i])
        elif count_joins == 2:
            joins_2_p.append(df[val][i])
            joins_2_e.append(df['tag'][i])       
        elif count_joins == 3:
            joins_3_p.append(df[val][i])
            joins_3_e.append(df['tag'][i])       
        elif count_joins == 4:
            joins_4_p.append(df[val][i])
            joins_4_e.append(df['tag'][i])       
        elif count_joins == 5:
            joins_5_p.append(df[val][i])
            joins_5_e.append(df['tag'][i])       
        else:
            joins_6_p.append(df[val][i])
            joins_6_e.append(df['tag'][i])       
    promedio = [joins_1_p, joins_2_p, joins_3_p, joins_4_p, joins_5_p, joins_6_p]
    errores = [joins_1_e, joins_2_e, joins_3_e, joins_4_e, joins_5_e, joins_6_e]
    return promedio, errores 


In [ ]:
original_p_joins, original_e_joins = split_by_joins(neo_df_final, 'real_old')
neo_p_joins, neo_e_joins = split_by_joins(neo_df_final,'pred_new')
ql_p_joins, ql_e_joins = split_by_joins(ql_df_final, 'pred_new')
sarsa_p_joins, sarsa_e_joins = split_by_joins(sarsa_df_final, 'pred_new')
exsarsa_p_joins, exsarsa_e_joins = split_by_joins(exsarsa_df_final, 'pred_new')

In [ ]:
len(exsarsa_p_joins[0])

In [ ]:

plt.rcParams.update({'font.size': 15})
# data to plot
n_groups = len(neo_p_joins)
    #igual = (neo_igual, ql_igual, sarsa_igual, exsarsa_igual)
    #mejor = (neo_mejor, ql_mejor, sarsa_mejor, exsarsa_mejor)
    #peor = (neo_peor, ql_peor, sarsa_peor, exsarsa_peor)

 # create plot
fig, ax = plt.subplots(figsize=(15,10))
index = np.arange(n_groups)
bar_width = 0.15
opacity = 0.8
c = ['blue','lightgreen','lightblue','yellow','orange']


mean_original = np.array([np.mean(val) for val in original_p_joins])
mean_neo = np.array([np.mean(val) for val in neo_p_joins])
mean_ql = np.array([np.mean(val) for val in ql_p_joins])
mean_sarsa = np.array([np.mean(val) for val in sarsa_p_joins])
mean_exsarsa = np.array([np.mean(val) for val in exsarsa_p_joins])


rects_original = plt.bar(index, mean_original, bar_width,
alpha=opacity,
color=c[0],
edgecolor = 'black',
label='VOS')
rects_neo = plt.bar(index+bar_width, mean_neo, bar_width,
alpha=opacity,
color=c[1],
edgecolor = 'black',
label='Neo')
rects_ql = plt.bar(index+bar_width*2, mean_ql, bar_width,
alpha=opacity,
color=c[2],
edgecolor = 'black',
label='Q-Learning')
rects_sarsa = plt.bar(index+bar_width*3, mean_sarsa, bar_width,
alpha=opacity,
color=c[3],
edgecolor = 'black',
label='Sarsa')
rects_exsarsa = plt.bar(index+bar_width*4, mean_exsarsa, bar_width,
alpha=opacity,
color=c[4],
edgecolor = 'black',
label='Expected Sarsa')


plt.xlabel('Número de JOINS')
plt.ylabel('Promedio de latencia')
plt.title(f'Latencia promedio con cierta cantidad de número de Joins para cada algoritmo')
plt.xticks(index+bar_width*2, ('1','2','3','4','5','6+'))
plt.legend()
plt.tight_layout()




plt.show()

### Consultas mejoradas, consultas iguales, consultas empeoradas diferenciados por numero de JOINs

In [ ]:
def plot_errores_join(lista_e_join,nombre_algoritmo):
    igual = []
    mejor = []
    peor = []
    for j in lista_e_join:
        i, m, p = j.count('igual'), j.count('mejor'), j.count('peor')
        igual.append(i)
        mejor.append(m)
        peor.append(p)
    plt.rcParams.update({'font.size': 13})
    # data to plot
    n_groups = len(igual)
    #igual = (neo_igual, ql_igual, sarsa_igual, exsarsa_igual)
    #mejor = (neo_mejor, ql_mejor, sarsa_mejor, exsarsa_mejor)
    #peor = (neo_peor, ql_peor, sarsa_peor, exsarsa_peor)

    # create plot
    fig, ax = plt.subplots(figsize=(15,10))
    index = np.arange(n_groups)
    bar_width = 0.15
    opacity = 0.8

    rects1 = plt.bar(index, mejor, bar_width,
    alpha=opacity,
    color='lightgreen',
    label='Mejor',
    edgecolor = 'black')

    rects2 = plt.bar(index + bar_width, igual, bar_width,
    alpha=opacity,
    color='khaki',
    label='Igual',
    edgecolor = 'black')

    rects3 = plt.bar(index + 2*bar_width, peor, bar_width,
    alpha=opacity,
    color='darksalmon',
    label='Peor',
    edgecolor = 'black')

    plt.xlabel('Número de JOINS')
    plt.ylabel('Promedio de latencia')
    plt.title(f'Número de errores por número de JOINS para algoritmo {nombre_algoritmo}')
    plt.xticks(index + bar_width, ('1','2','3','4','5','6+'))
    plt.legend()
    plt.tight_layout()


    for i in index:
        ax.text(index[i], 
                 mejor[i]*0.7,
                 str(mejor[i]),
                 ha='center', va='bottom',
                )
        ax.text(index[i] + bar_width, 
                 igual[i]*0.75,
                 str(igual[i]),
                 ha='center', va='bottom',
                )
        ax.text(index[i] + 2*bar_width, 
                 peor[i]*0.75,
                 str(peor[i]),
                 ha='center', va='bottom',
                )
    plt.show()

In [ ]:
plot_errores_join(neo_e_joins,'NEO')
plot_errores_join(ql_e_joins,'Q-Learning')
plot_errores_join(sarsa_e_joins,'Sarsa')
plot_errores_join(exsarsa_e_joins,'Expected Sarsa')

### RMSE y MAE Por iteración

In [ ]:
#ds_rl
#neo_df_final
#ql_df_final
#sarsa_df_final
#exsarsa_df_final

In [ ]:
neo_df_no_history[['mse_new','mse_old']]